We shall use a Linear SVM classifier on the dataset r1 and compare the results with the CNN based models.

In [31]:
import skimage as ski
import numpy as np
from torch.utils.data import Dataset
import os
import pandas as pd
from torch.utils.data import DataLoader
import time
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Data loading

In [16]:
class PetDatasetSVM(Dataset):
    def __init__(self, annotations_file, img_dir):
        self.img_annotations = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        
    def __len__(self):
        return len(self.img_annotations)
    
    def __getitem__(self,idx):
        img_path = os.path.join(self.img_dir,self.img_annotations.iloc[idx,0])
        image = ski.io.imread(img_path)/255
        image = ski.transform.rescale(image,0.85)    # resize to 255 pix a side
        image = image[:,:,:3]  # dropping the alpha channel if any
        image = image.reshape(-1) # reshaping the image data into a 1d vector
        class_label = self.img_annotations.iloc[idx,2]-1      # the -1 is coz labels are expected from 0 to C-1
            
        return image, class_label

In [17]:
train_r1 = PetDatasetSVM('annotations_aug/annotations_train_r1.csv','img_augmented_processed/')
valid = PetDatasetSVM('annotations_aug/annotations_valid.csv','img_augmented_processed/')
test = PetDatasetSVM('annotations_aug/annotations_test.csv','img_augmented_processed/')

In [22]:
all_data = len(train_r1)
train_r1_loader = DataLoader(train_r3,batch_size=all_data)
train_images, train_labels = next(iter(train_r1_loader))
train_images, train_labels = train_images.numpy(), train_labels.numpy() # Dataloader outputs a tensor, converting to numpy ndarray


valid_loader = DataLoader(valid,batch_size=len(valid))
valid_images, valid_labels = next(iter(valid_loader))
valid_images, valid_labels = valid_images.numpy(), valid_labels.numpy()

test_loader = DataLoader(test,batch_size=len(test))
test_images, test_labels = next(iter(test_loader))
test_images, test_labels = test_images.numpy(), test_labels.numpy()

In [25]:
print('shape of training batch (images): {}'.format(train_images.shape))
print('shape of training batch (labels): {}'.format(train_labels.shape))
type(train_images)

shape of training batch (images): (7102, 195075)
shape of training batch (labels): (7102,)


numpy.ndarray

# SVM model

In [27]:
feature_scale = StandardScaler()
train_images = feature_scale.fit_transform(train_images)      # setting mean to 0, variance to 1
valid_images = feature_scale.fit_transform(valid_images) 
test_images = feature_scale.fit_transform(test_images) 

In [30]:
svm_trainer = SVC()
svm_trainer.fit(train_images, train_labels)

SVC()

In [32]:
train_r1_loader2 = DataLoader(train_r3,batch_size=2048, shuffle=True)
train_images_test, train_labels_test = next(iter(train_r1_loader2))
train_images_test, train_labels_test = train_images_test.numpy(), train_labels_test.numpy()

train_predictions = svm_trainer.predict(feature_scale.fit_transform(train_images_test))
print('Accuracy on a random training batch:{}'.format(accuracy_score(train_labels_test,train_predictions)))


Accuracy on a random training batch:0.40673828125


In [33]:
start = time.time()

train_images_test, train_labels_test = next(iter(train_r1_loader2))
train_images_test, train_labels_test = train_images_test.numpy(), train_labels_test.numpy()

train_predictions = svm_trainer.predict(feature_scale.fit_transform(train_images_test))
print('Accuracy on a random training batch:{}'.format(accuracy_score(train_labels_test,train_predictions)))

end = time.time()
print('Time taken:{}'.format(end-start))

Accuracy on a random training batch:0.40771484375
Time taken:2202.200286626816


In [34]:
start = time.time()

valid_predictions = svm_trainer.predict(valid_imagesss)
print('Accuracvalid_imagesmagesmagese validation set:{}'.format(accuracy_score(valid_labels,valid_predictions)))

end = time.time()
print('Time taken:{}'.format(end-start))

Accuracy on the validation set:0.07377598926894702
Time taken:1592.6651875972748


This is much worse than the CNN based models.